In [2]:
import subprocess 
import sys
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'gensim'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'sentence_transformers'])
import gensim
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from tqdm import tqdm
from sklearn import preprocessing
from gensim.parsing.preprocessing import preprocess_string
from gensim.parsing.preprocessing import strip_tags       # strip html tags
from gensim.parsing.preprocessing import strip_multiple_whitespaces
from gensim.parsing.preprocessing import stem_text
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import strip_punctuation, strip_non_alphanum
from sentence_transformers import SentenceTransformer
import nltk
import matplotlib.pyplot as plt
import numpy as np
from sklearn.feature_selection import VarianceThreshold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier

In [3]:
def apply_filters(text):
    result = []
    filters = [lambda x: x.lower(), stem_text, strip_punctuation, strip_non_alphanum, remove_stopwords,
               strip_multiple_whitespaces]
    for sent in text:
        parsed_line = preprocess_string(sent, filters)
        result.append(parsed_line)
    return sum(result, [])


def preprocessing_fun(df, token=True):
    sent_detector = nltk.data.load('punkt/english.pickle')
    # Preprocessing filters
    if token:
        df['preprocessed'] = df.apply(lambda row: sent_detector.tokenize(row['body']), axis=1).apply(apply_filters)
    else:
        df['preprocessed'] = df.apply(lambda row: sent_detector.tokenize(row['body']), axis=1)
    return df

In [4]:
df_biden_train = pd.read_csv('./comments/JoeBiden_train.csv').rename(columns={"Unnamed: 0": "index"}).set_index("index")
df_biden_train['subreddit'] = 0
df_biden_test = pd.read_csv('./comments/JoeBiden_test.csv').rename(columns={"Unnamed: 0": "index"}).set_index("index")
df_biden_test['subreddit'] = 0
df_trump_train = pd.read_csv('./comments/The_Donald_train.csv').rename(columns={"Unnamed: 0": "index"}).set_index("index")
df_trump_train['subreddit'] = 1
df_trump_test = pd.read_csv('./comments/The_Donald_test.csv').rename(columns={"Unnamed: 0": "index"}).set_index("index")
df_trump_test['subreddit'] = 1

df_train = pd.concat([df_biden_train, df_trump_train], axis=0)
df_test = pd.concat([df_biden_test, df_trump_test], axis=0)

In [5]:
df_train = df_train.sample(frac=1).reset_index().drop(columns="index")
df_test = df_test.sample(frac=1).reset_index().drop(columns="index")

# Preprocessing
df_train = preprocessing_fun(df_train, token=True)
df_test = preprocessing_fun(df_test, token=True)
nbr_test = len(df_test)

In [ ]:
df_tot = df_train.append(df_test)
tqdm.pandas()
sent_detector = nltk.data.load('punkt/english.pickle')
embedder = SentenceTransformer('roberta-base-nli-stsb-mean-tokens')
encoded = df_tot['preprocessed'].progress_apply(lambda row: embedder.encode(row))

/Users/margauxgerard/Documents/Mémoire 2021/CovidThesis/venv/lib/python3.7/site-packages/tqdm/std.py:670: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
  4%|▍         | 1110/28214 [06:01<1:52:59,  4.00it/s]

In [1]:
list_df = encoded.iloc[0]['preprocessed'].reshape(-1)
with tqdm(total=len(range(1, encoded.shape[0]-1))) as pbar:
    for i in range(1, encoded.shape[0]-1):
        df = encoded.iloc[i]['preprocessed'].reshape(-1)
        list_df.append(df)
        pbar.update(1)
df_tot = pd.DataFrame(list_df)
df_tot

NameError: name 'encoded' is not defined